# Autoregressive models using a feedforward neural network 
## PART 2: Applying the methods to health care time series

In this notebook we will use a feedforward neural network to fit a single and ensemble linear and non-linear models to real time series data. 

<div class="alert alert-info">
    
1. Most of the work we will do is data manipulation: preprocessing data and making sure it is the right shape for the neural networks.

2. The ensemble learning method can be computationally expensive.  We have included some pre-trained models that can be loaded from file if needed.
</div>

---
**LEARNING OBJECTIVES**

* Learn how to apply feedforward neural networks to real health data.
* Methods to preprocess nn input data.
* Recognise the stochastic nature of neural network training
* Use a ensemble of neural networks to provide a more reliable point forecast
---


# Python dependencies

It is recommended that you use the forecasting course conda environment provided. We are again going to implement neural networks using `tensorflow` and '`keras`. You should be using at least `tensorflow` version `2.1.0`.

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping

from statsmodels.tools.eval_measures import rmse

tf.__version__

# Forecasting emergency admissions in England

We will now use feedforward neural networks to predict the number of monthly emergency admissions in England. 

## Load the data

**Task**:
* Execute the code below to read the emergency admissions data into pandas

In [ ]:
url = 'https://raw.githubusercontent.com/health-data-science-OR/data/master' \
        + '/em_admits_ts.csv'
em_admits = pd.read_csv(url)

In [ ]:
em_admits.head(3)

In [ ]:
em_admits.shape

## 2.3. Preprocessing 

### 2.3.1 Datetime format

Notice the the `month_year` column in `em_admits` holds a string an invalid date format e.g. 'Aug-10'.  Pandas cannot handle this as-is because '10' could refer to any century!  So let's do a bit of preprocessing to get it into a valid datetime format.


*Optional Task:*
* Take some time to understand the code that preprocesses the dates.  This is real health data and it is likely you will need to deal with formatting issues as experienced here.

First we will format the string to something pandas can parse i.e. 'Aug 2010'.  Then we will call the `pd.to_datetime()` function to parse the string and return a `datetime`.  We will assign the result to our dataframe's index and set the freq to monthly start 'MS'

In [ ]:
date_str = em_admits['month_year'].str[:3] + ' 20' \
    + em_admits['month_year'].str[-2:]
date_str.name = 'date'
em_admits = em_admits.set_index(pd.to_datetime(date_str))
em_admits.index.freq = 'MS'
em_admits = em_admits.drop(columns=['month_year'])

In [ ]:
em_admits.head()

## Visualise the training data

We will be forecasting the last 12 months of the series.  Let's take a look at the training data (being careful to exclude the last 12 months)

In [ ]:
holdout_length = 12
em_admits[:len(em_admits)-holdout_length].plot(figsize=(12,4));

## Calender adjustment

This is monthly data so a useful preprocessing step is to transform the data into a daily rate by dividing by the number of days in the month. When we plot this the troughs we saw in Feb each year disappear.

**execute the code below which:**:
* Calculates the average admissions per day series
* Plots the training data (holding back 12 months for testing)

In [ ]:
admit_rate = em_admits['em_admits'] / em_admits.index.days_in_month

In [ ]:
admit_rate[:len(admit_rate)-12].plot(figsize=(12,4));

# **Exercise 1**: Convert the time series to format suitable for supervised learning.

The function `sliding_window` has been provided below for you to create your training data.

**Task**:
* Using a sliding window approach convert the time series into a tabular format. 
 * Use a window size of 12 and assume you are predicting a scalar value of y (1-step ahead).
* Conduct a train test split holding back 12 windows as a validation set.

In [ ]:
def sliding_window(train, window_size=2, horizon=1):
    '''
    sliding window.
    
    Parameters:
    --------
    train: array-like
        training data for time series method
    
    window_size: int, optional (default=2)
        lookback - how much lagged data to include.
        
    horizon: int, optional (default=1)
        number of observations ahead to predict
            
    Returns:
        array-like, array-like
    
        preprocessed X, preprocessed Y
    '''
    tabular_X = []
    tabular_y = []
    
    for i in range(0, len(train) - window_size - horizon):
        X_train = train[i:window_size+i]
        y_train = train[i+window_size+horizon-1]
        tabular_X.append(X_train)
        tabular_y.append(y_train)
       
    return np.asarray(tabular_X), np.asarray(tabular_y).reshape(-1, )

In [ ]:
# your code here...

# get data in tabular format for NN
#X_data, y_data = sliding_window(...)

#X_train, y_train, X_test, y_test = ... train test split code

# Scaling the features and target to be between -1 and 1
In many machine learning applications data are scaled to be between 0 and 1. For neural network forecasting, *Ord, Fildes and Kourentzes (2017)* recommend scaling to be between -1 and 1.  This is what we will do here.  To do the scaling we will use

```python
sklearn.preprocessing.MinMaxScaler
```

> Execute the code below to transform the data.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))

# scale on training data
scaler.fit(admit_rate.iloc[:-12].to_numpy().reshape(-1, 1))

In [ ]:
# what is the shape of X_train
X_train.shape

In [ ]:
# peak at first sample in X_train
X_train[0]

In [ ]:
def scale_arrays_of_X(X_to_scale, scaler):
    """
    Takes a 2d array and transforms data using a sklearn scaler.  
    This is executed in a loop as the sklearn scalers have a strict 
    interface for the dimensions of X.
    
    Params:
    ------
    X_to_scale: np.ndarray
        Sliding windows of X data
        
    scaler: MinMaxScaler
        Fitted scaler used to transform X_train.
        
    Returns:
    -------
    np.ndarray 
        Dimensions are the same as X_train.
    """
    X_scaled = []
    
    # scale each sample separately
    for sample in X_to_scale:
        # scale the sample
        scaled_sample = scaler.transform(sample.reshape(-1, 1))
        
        # store the sample in a list
        X_scaled.append(scaled_sample)
    
    # return the scaled data as np.ndarray with dimensions from input
    return np.array(X_scaled).reshape(len(X_scaled), -1)

In [ ]:
# scale all data
y_train = scaler.transform(y_train.reshape(-1, 1))
X_train = scale_arrays_of_X(X_train, scaler)
X_test = scale_arrays_of_X(X_test, scaler)
y_test = scaler.transform(y_test.reshape(-1, 1))

In [ ]:
# is the shape of X_train correct? yes.
X_train.shape

In [ ]:
# peak at first sample after transform.
X_train[0]

# **Exercise 2**: A Linear regression model benchmark

The first model we will try is the linear model. Its will serve as our neural network baseline.  (In practice we would also check this is better than a naive method such as seasonal naive).

## Exercise 2a Train the model

**Task:**
* Using `Keras`, construct a neural network that mimics a simple linear regression model (see previous notebook).  
* Optional: To get comparable results, set the tensorflow random number seed to 1234
* Train the model for 100 epochs.
* Optionally you can use an early stopping callback with patience set to 10.

In [ ]:
# your code here ...

def get_linear_model(ws):
    '''
    Sequential Keras model that minics
    AR linear model. 
    
    '''
    pass

## Exercise 2b. Generate and evaluate a multi-step forecast

**Task:**
* Using the iterative method produce a 12 step forecast. Save the predictions in a variable called `y_preds_lm`
* Calculate the RMSE
* Optional: Plot the results -> predictions versus validation.

**Hints:**
* A function `autoregressive_iterative_forecast` is provided below.  (you could use this function or write your own if you prefer!)

In [ ]:
def autoregressive_iterative_forecast(model, exog, h):
    '''
    h-step forecast for an autoregressive 
    model using the iterative prediction method.
    
    Conduct h one-step forecasts gradually
    replacing ground truth autoregressive X 
    values with predictions.
    
    Parameters:
    ------
    model: forecast object
        model that has a .predict(h) interface
        
    exog: array-like
        initial vector of lagged values (X)
    
    h: int
        forecast horizon. assumed to be > 0
    
    Returns:
    ------
    numpy.ndarray
        y_predictions
    '''
    y_preds = []
    current_X = exog
    for i in range(h):
        y_pred = model.predict(current_X.reshape(1, -1))[0,0]
        y_preds.append(y_pred)

        current_X = np.roll(current_X, shift=-1)
        current_X[-1] = y_pred

    return np.array(y_preds)

In [ ]:
##### your code here ...

# **Exercise 3:** Training a non-linear deep network

Now that you have got the basic structure and mechanics of the code you need for forecasting let's build a more complex model and compare the RMSE on the validation set to your simple linear model.

**Task:** 
* Create a new neural network model with 2 hidden layers
* Try 32 and 64 neurons for layer 1 and 2 respectively
* Use a ReLU activation function.
* Use the Adam optimiser with a learning rate of 0.01
* Predict the next 12 months ahead
* Calculate the RMSE

**Hints:**
* Feel free to experiment with the number of hidden layers, neurons and learning rate.
* Perhaps try a dropout layer(s) if you feel your model is overfitting.
* Set a tensorflow random seed if you want to be able to reproduce your results e.g. 45676
* To organise your code you may want to create a function that creates the Keras model. This could, for example, be called `get_network_model`



In [ ]:
# your code here ...

# Ensemble Learning

In all of the examples above we have been setting a random seed for tensorflow.  This 'suggests' that if we used a different randon number seed we would get a slightly different result (this is due to both random initialisation of weights/biases and stochastic gradient descent). Neural networks are extremely flexible and have many parameters. This leads to one of the key challenges with neural networks - overfitting.  There are multiple ways to deal with overfitting.  In forecasting a common approach is to use an **ensemble** of models.  

In an ensemble we train multiple models. 

## Training an ensemble

We will train an ensemble of neural networks that mimic a linear model.  

The code below has been provided for you to work through.

* We set some parameters e.g. number of models in an the ensemble: 20 to 30 should be plenty.
* We use a python loop to create and train each model and store the model in a python list.
    * Optionally we can save the models to file and load pre-trained versions at a later date.
* To predict we the need to loop through the collection of models.

In [ ]:
def load_pretrained_ensemble(n_models):
    '''
    Load the pre-trained ensemble models (only use if they exist!)
    '''
    models = []
    url = '/input'
    for n in range(n_models):
        model_n = tf.keras.models.load_model(f'{url}/ensemble_model_{n}.h5')
        models.append(model_n)
    return models

In [ ]:
# script to train the models.

################# Parameters for the ensemble #################################
#set random seed so that ensemble can be repeated.
tf.random.set_seed(1085)

#number of models to create...
N_MODELS = 20

#max no. of epochs for training of each model.
N_EPOCHS = 100

#no. of autoregressive lags
WINDOW_SIZE = 12

#early stopping reguluarization
es = EarlyStopping(monitor='loss', patience=10)

#I've pretrained 50 models you can load them from file if wanted.
LOAD_FROM_FILE = False

###############################################################################

if LOAD_FROM_FILE:
    #it will take a few seconds to load.
    models = load_pretrained_ensemble(N_MODELS)
else:
    models = []
    for n in range(N_MODELS):
        # single layer nn
        model_n = get_linear_model(WINDOW_SIZE)

        # fit model silently (verbose=0)
        history = model_n.fit(x=X_train, 
                              y=y_train, 
                              epochs=N_EPOCHS,
                              verbose=0, 
                              callbacks=[es], 
                              batch_size=32)

        # this will overwrite pre-trained models.
        model_n.save(f'input/ensemble_model_{n}.h5')
        models.append(model_n)

### Predictions in an ensemble

In an ensemble, we predict in a loop. In python this is straightfoward as we simply loop through the models we have trained and call `autoregressive_iterative_forecast`. We will store the predictions of each forecast in a python `list`  called `e_preds`

<div class="alert alert-info">
    
In an ensemble we end up with a distribution of forecasts!  For point forecasts we could then take the median of the forecasts.  We can also get a measure of variability in the forecasts by calculating the quantiles. 
    
</div>



In [ ]:
# create the forecasts
# this code will take a few seconds to execute
H = 12
e_preds = []
for model in models:
    y_preds = autoregressive_iterative_forecast(model, X_test[0], h=H)
    e_preds.append(y_preds)
    
e_preds = np.array(e_preds)

Inverse transform the data and calculate the median and 0.025 and 0.975 percentiles of the point forecasts

Remember we can use `scaler.inverse_transform()`

In [ ]:
e_preds = np.asarray(e_preds)
e_preds_tran = scaler.inverse_transform(e_preds).T
y_preds_mdn = np.percentile(e_preds_tran.T, 50, axis=0)
y_preds_2_5 = np.percentile(e_preds_tran.T, 2.5, axis=0)
y_preds_97_5 = np.percentile(e_preds_tran.T, 97.5, axis=0)
y_preds_mdn.shape

In [ ]:
# plot the individual forecasts and the median

fig,ax = plt.subplots(1, 2, sharey=True, figsize=(12, 4))
ax[0].plot(e_preds_tran)
ax[0].plot(scaler.inverse_transform(y_test), label='test', linestyle='--', 
         color='red')
ax[0].plot(y_preds_mdn, label='median', linestyle='-', color='black')
ax[0].legend()
ax[0].set_title(f'Point forecasts: {N_MODELS} models')

ax[1].plot(scaler.inverse_transform(y_test), label='test', linestyle='--', 
         color='red')
ax[1].plot(y_preds_mdn, label='median', linestyle='-', color='black')
ax[1].plot(y_preds_2_5, label='0.025 percentile', linestyle='-.', color='black')
ax[1].plot(y_preds_97_5, label='0.975 percentile', linestyle='--', color='black')
#ax[1].plot(y_preds_lm, label='original lmforecast', linestyle='--', color='green')
ax[1].set_title(f'Middle 95% of point forecasts ')
ax[1].legend();

In [ ]:
rmse_lm = rmse(scaler.inverse_transform(y_test), y_preds_lm)[0]
rmse_mdn = rmse(scaler.inverse_transform(y_test), y_preds_mdn)[0]

print(f'rmse lm: {rmse_lm:.2f}\nrmse ensemble: {rmse_mdn:.2f}')

In [ ]:
rmse_25 = rmse(scaler.inverse_transform(y_test), y_preds_2_5)[0]
rmse_75 = rmse(scaler.inverse_transform(y_test), y_preds_97_5)[0]
print(f'95% of linear models will have rmse between: {rmse_75:.2f} - {rmse_25:.2f}')

**Question**: Is the ensemble approach useful?  What does it tell us about our original linear model?

## Exercise 4: Create an ensemble of non-linear models.

Is the two layer model more accurate than the simple linear regression model and its ensemble counterpart?

**Task:** 

* Create an ensemble of 20 models.
* Each model should be based on your solution to exercise 2 (e.g. a neural network with 2 hidden layers)
* Optional: save your models to file. (recommended)
* Forecast the next 12 periods.
* Calculate the RMSE of the forecast.


**Hints:**

* You have **all of the code** you need to complete this task!
* Remember to back transform your forecasts
* Use the median of the ensemble.
* Look carefully at the previous ensemble example.

**Questions**
* Which out of the simple linear, multi-layer and ensemble models do you think is best in this instance?

In [ ]:
# your code here ...

## Optional Extra exercise for you to think about.
* How would you use a ensemble method with a model that predicts a vector?

# End of lab